In [114]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [115]:
adr_info_filename = '../data/raw/adr_info.csv'
adr_info = pd.read_csv(adr_info_filename)

adr_dict = adr_info[['id','adr']].set_index('id')['adr'].str.replace(' US Equity', '').to_dict()
asia_tickers = adr_info[adr_info['country'].isin(['JAPAN','AUSTRALIA'])]['adr'].str.replace(' US Equity','').tolist()

In [116]:
adr_close_df = pd.read_csv('../data/raw/adrs/adr_PX_LAST_adjust_all.csv', index_col=0, parse_dates=True)
adr_open_df = pd.read_csv('../data/raw/adrs/adr_PX_OPEN_adjust_all.csv', index_col=0, parse_dates=True)
ord_open_df = pd.read_csv('../data/processed/ordinary/ord_open_to_usd_adr_PX_OPEN_adjust_all.csv', index_col=0, parse_dates=True)
ord_close_df = pd.read_csv('../data/processed/ordinary/ord_close_to_usd_adr_PX_LAST_adjust_all.csv', index_col=0, parse_dates=True)

ord_open_df = ord_open_df.rename(columns=adr_dict)[asia_tickers].dropna(how='all')
ord_close_df = ord_close_df.rename(columns=adr_dict)[asia_tickers].dropna(how='all')

In [121]:
# ord_open_df['MFG']

In [102]:
((ord_close_df / ord_open_df) - 1).std()

TAK     0.010015
SONY    0.012956
TM      0.012064
HMC     0.012400
MUFG    0.013166
SMFG    0.011972
MFG     0.012175
BHP     0.011279
WDS     0.014616
dtype: float64

In [103]:
((adr_close_df[asia_tickers] / adr_open_df[asia_tickers]) - 1).std()

TAK     0.009066
SONY    0.010609
TM      0.007705
HMC     0.008395
MUFG    0.009704
SMFG    0.009551
MFG     0.010553
BHP     0.011164
WDS     0.014198
dtype: float64

In [104]:
# ((ord_close_df / ord_ord_df) - 1).std()

In [105]:
stacked_ord = ord_close_df.stack().rename('ord_close')
stacked_ord.index.rename(['date','ticker'], inplace=True)

stacked_adr = adr_open_df.stack().rename('adr_open')
stacked_adr.index.rename(['date','ticker'], inplace=True)

ord_close_to_adr_open = pd.merge(stacked_ord, stacked_adr, left_index=True, right_index=True)

In [106]:
stacked_adr = adr_close_df.stack().rename('adr_close')
stacked_adr.index.rename(['date','ticker'], inplace=True)

stacked_ord = ord_open_df[ord_open_df.index.diff() == pd.Timedelta('1D')].shift(1).stack().rename('ord_open')
stacked_ord.index.rename(['date','ticker'], inplace=True)

adr_close_to_ord_open = pd.merge(stacked_adr, stacked_ord, left_index=True, right_index=True)

In [107]:
for ticker in asia_tickers:
    ord_return = ord_close_df[ticker] / ord_open_df[ticker] - 1
    adr_return = adr_close_df[ticker].pct_change()
    print(ticker, adr_return.loc['2025':].corr(ord_return.loc['2025':]))

TAK 0.4245535794434293
SONY 0.4784007713087649
TM 0.6012058092817806
HMC 0.5872437847924866
MUFG 0.6230350232730908
SMFG 0.528060891332601
MFG 0.6256261412944574
BHP 0.32783801264659407
WDS 0.5948141289235822


In [108]:
adr_close_to_ord_open.pct_change(axis=1).loc['2025':].groupby('ticker').std()

,adr_close,ord_open
ticker,,
BHP,NaN,0.027083
HMC,NaN,0.026067
MFG,NaN,0.030582
MUFG,NaN,0.027494
SMFG,NaN,0.027768
SONY,NaN,0.028281
TAK,NaN,0.019666
TM,NaN,0.025826
WDS,NaN,0.032426


In [109]:
ord_close_to_adr_open.pct_change(axis=1).loc['2025':].groupby('ticker').std()

,ord_close,adr_open
ticker,,
BHP,NaN,0.009863
HMC,NaN,0.007090
MFG,NaN,0.007837
MUFG,NaN,0.007735
SMFG,NaN,0.007527
SONY,NaN,0.008114
TAK,NaN,0.006363
TM,NaN,0.006000
WDS,NaN,0.007173


In [110]:
adr_close_to_ord_open.pct_change(axis=1).loc['2025':].groupby('ticker').std()

,adr_close,ord_open
ticker,,
BHP,NaN,0.027083
HMC,NaN,0.026067
MFG,NaN,0.030582
MUFG,NaN,0.027494
SMFG,NaN,0.027768
SONY,NaN,0.028281
TAK,NaN,0.019666
TM,NaN,0.025826
WDS,NaN,0.032426


In [111]:
adr_df = pd.read_csv('../data/raw/adrs/adr_PX_LAST_adjust_all.csv', index_col=0, parse_dates=True)
under_df = pd.read_csv('../data/processed/ordinary/ord_open_to_usd_adr_PX_OPEN_adjust_all.csv', index_col=0, parse_dates=True).shift(1)
holdings_filename = '../data/raw/adr_info.csv'
hold_df = pd.read_csv(holdings_filename)

adr_dict = hold_df[['id','adr']].set_index('id')['adr'].str.replace(' US Equity', '').to_dict()
under_df = under_df.rename(columns=adr_dict)

adj_prem_df = ((adr_df.div(under_df)).dropna(axis=0, how='all')-1)*1e4
asia_tickers = hold_df[hold_df['country'].isin(['JAPAN','AUSTRALIA'])]['adr'].str.replace(' US Equity','').tolist()

In [112]:
adr_df = pd.read_csv('../data/raw/adrs/adr_PX_OPEN_adjust_all.csv', index_col=0, parse_dates=True)
under_df = pd.read_csv('../data/processed/ordinary/ord_close_to_usd_adr_PX_LAST_adjust_all.csv', index_col=0, parse_dates=True)

holdings_filename = '../data/raw/adr_info.csv'
hold_df = pd.read_csv(holdings_filename)

adr_dict = hold_df[['id','adr']].set_index('id')['adr'].str.replace(' US Equity', '').to_dict()
under_df = under_df.rename(columns=adr_dict)

adj_prem_df2 = ((adr_df.div(under_df)).dropna(axis=0, how='all')-1) * 1e4
asia_tickers = hold_df[hold_df['country'].isin(['JAPAN','AUSTRALIA'])]['adr'].str.replace(' US Equity','').tolist()

In [113]:
adr_df = pd.read_csv('../data/processed/adrs/adr_open_mid.csv', index_col=0, parse_dates=True)[asia_tickers]
under_df = pd.read_csv('../data/processed/ordinary/ord_close_to_usd_adr_PX_LAST_adjust_none.csv', index_col=0, parse_dates=True)

holdings_filename = '../data/raw/adr_info.csv'
hold_df = pd.read_csv(holdings_filename)

adr_dict = hold_df[['id','adr']].set_index('id')['adr'].str.replace(' US Equity', '').to_dict()
under_df = under_df.rename(columns=adr_dict)

adj_prem_df3 = ((adr_df.div(under_df)).dropna(axis=0, how='all')-1) * 1e4
asia_tickers = hold_df[hold_df['country'].isin(['JAPAN','AUSTRALIA'])]['adr'].str.replace(' US Equity','').tolist()

In [86]:
for t in asia_tickers:
    if t in adj_prem_df.columns:
        #plt.figure()
        #adj_prem_df[t].loc['2020':].plot(label=t)
        #plt.title(f'Adjusted Premium for {t}')
        print(t, adj_prem_df[t].loc['2020':].mean())

TAK 2.9070731740970452
SONY 73.085434472489
TM 7.930910278608186
HMC 10.383686100257327
MUFG 30.79292167973841
SMFG 21.376652530975996
MFG 70.05757555081173
BHP 11.299863310158814
WDS -15.465481083466285


In [87]:
for t in asia_tickers:
    if t in adj_prem_df2.columns:
        # plt.figure()
        # adj_prem_df2[t].loc['2020':].plot(label=t)
        # plt.title(f'Adjusted Premium for {t}')
        print(t, adj_prem_df2[t].loc['2020':].mean())

TAK 0.8517066806462611
SONY 62.47765489933873
TM 11.571787161173434
HMC 13.359582490860086
MUFG 15.364059380827321
SMFG 7.834951978316122
MFG 56.76265962975671
BHP 0.9316396863099604
WDS -15.10389561840794


In [90]:
for t in asia_tickers:
    if t in adj_prem_df3.columns:
        # plt.figure()
        # adj_prem_df2[t].loc['2020':].plot(label=t)
        # plt.title(f'Adjusted Premium for {t}')
        print(t, adj_prem_df3[t].loc['2025':].mean())

TAK -7.822054058355295
SONY -17.339110178666914
TM -7.63034657020709
HMC -11.02393263486262
MUFG 13.929829311274144
SMFG 17.466012296486696
MFG 50.772653980864874
BHP -3.0179760319277578
WDS 5.68011701858708


In [98]:
# adj_prem_df2['SONY'].loc['2025':].plot()

In [ ]:
div_df = pd.read_csv('../data/raw/all_dividends.csv', index_col=0, parse_dates=True)
adr_df.stack().rename('adr')